In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

/home/csci4850/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# need to get data setup and organized (it is mostly)
data = []
one_hot_index = 0
reviewers = {}
y_training = []

# need to find out exactly how many unique reviewerIDs there are;
# that will replace 100 in np.zeros, since that will be the output of the final softmax layer

authors = 100

with open('file2.out') as file:
    for line in file:
        split = line.split('\t')
        vectorVals = split[0][1:-1].split(',')
        reviewerID = split[1].strip()
        if reviewerID not in reviewers:
            reviewers[reviewerID] = np.zeros(authors)
            reviewers[reviewerID][one_hot_index] = 1
            one_hot_index += 1
            y_training.append(reviewers[reviewerID])
        else:
            y_training.append(reviewers[reviewerID])
        data.append([float(x) for x in vectorVals])
        # need to go thru file to find out how many unique authors, capped at 100 for testing a small sample set
        if one_hot_index == 100:
            break
print(len(data))
print(len(data[0]))

# =====================================================
# =====================================================
x_training = np.array(data)
# convert the 2d to 3d representation
x_training = x_training.reshape(x_training.shape + (1,))
output_shape = x_training.shape[1]
input_shape = x_training.shape[2]
y_training = np.array(y_training)
print('X_training shape: ' + str(x_training.shape))
# =====================================================
# =====================================================

print('='*16)
print(len(y_training))
print(len(y_training[0]))

144
300
X_training shape: (144, 300, 1)
144
100
Duplicates: 44


In [3]:
model = keras.Sequential()

model.add(keras.layers.Conv1D(300, kernel_size=(3), strides=(1),
                 activation='relu',
                 input_shape=[output_shape,
                              input_shape]))

model.add(keras.layers.SimpleRNN(20, activation='tanh', use_bias=True, 
                       kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                       bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, 
                       bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
                       recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.15, 
                       return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False))

model.add(keras.layers.Dropout(.20, noise_shape=None, seed=None))
model.add(keras.layers.Dense(100, activation='relu'))
#model.add(keras.layers.GlobalMaxPool1D()) 
model.add(keras.layers.Dense(y_training.shape[1], activation='softmax'))

#model.compile(loss=keras.losses.sparse_categorical_crossentropy,
#    optimizer=keras.optimizers.Adam(),
#    metrics=['accuracy'])

model.compile(loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 296, 300)          1800      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 20)                6420      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
Total params: 20,420
Trainable params: 20,420
Non-trainable params: 0
_________________________________________________________________


In [4]:
#y_train = np.zeros(1)
#y_train[0] = 1
y_training = np.array(y_training)
#y_train = np.array([y_train])
#print(y_training)
#newArray = np.array([newArray])
#y_training = y_training.reshape((1,) + y_training.shape)
#newArray = np.array([newArray])
'''
print('X_TRAINING ARRAY')
print()
print(x_training.shape)
print(x_training[0])

print()
print('Y_TRAINING ARRAY')
print()
print(y_training.shape)
print(y_training[0])
'''
batch_size = 20
epochs = 1000
history = model.fit(x_training, y_training,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1, #)
    validation_split = 0.2)


Train on 115 samples, validate on 29 samples
Epoch 1/1000
115/115 [==============================] - 3s 26ms/step - loss: 4.6097 - acc: 0.0087 - val_loss: 4.6684 - val_acc: 0.0345
Epoch 2/1000
115/115 [==============================] - 2s 19ms/step - loss: 4.4952 - acc: 0.0261 - val_loss: 4.7252 - val_acc: 0.0000e+00
Epoch 3/1000
115/115 [==============================] - 2s 19ms/step - loss: 4.4293 - acc: 0.0696 - val_loss: 4.7969 - val_acc: 0.0000e+00
Epoch 4/1000
115/115 [==============================] - 2s 19ms/step - loss: 4.3699 - acc: 0.0870 - val_loss: 4.8514 - val_acc: 0.0345
Epoch 5/1000
115/115 [==============================] - 2s 19ms/step - loss: 4.3271 - acc: 0.0609 - val_loss: 4.9397 - val_acc: 0.0345
Epoch 6/1000
115/115 [==============================] - 2s 19ms/step - loss: 4.2279 - acc: 0.0609 - val_loss: 5.0317 - val_acc: 0.0345
Epoch 7/1000
115/115 [==============================] - 2s 20ms/step - loss: 4.1511 - acc: 0.0696 - val_loss: 5.1641 - val_acc: 0.0000e+0

In [1]:
#score = model.evaluate(x_testing, y_testing, verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

## Citations